In [1]:
import pickle
import os
import glob
import torch
import numpy as np
import pandas as pd
import scipy.io
from sklearn.preprocessing import MinMaxScaler
from matplotlib import pyplot as plt
from scipy import signal
from scipy import stats
import seaborn as sns
import statsmodels.api as sm
from tqdm import tqdm
from tqdm.notebook import tqdm_notebook
tqdm_notebook.pandas()

# jupyter notebook display setting for all data structures
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
pd.set_option('display.width', None)
np.set_printoptions(threshold=np.inf)

In [2]:
# define the path to the data file
data_file = '/project2/xinhe/waqaas/DNA-breathing/data/Chipseq_data/seq_breathing_feat.pkl'

# define path to the TF index
tf_index_file = '/project2/xinhe/waqaas/DNA-breathing/data/Chipseq_data/group_index.pt'

# Preliminary data prep

cutting out flanks

In [3]:
with open(data_file, 'rb') as f:
    panset = pickle.load(f)

for part in panset.keys():
    partition = panset[part]
    for seq_id in partition.keys():
        sample = partition[seq_id]
        # remove the first 150 and last 150 bp from seq, coord, coord_sq, flip
        sample['seq'] = sample['seq'][150:-150]
        sample['coord'] = sample['coord'][150:-150]
        sample['coord_sq'] = sample['coord_sq'][150:-150]
        sample['flip'] = sample['flip'][150:-150]

for part in panset.keys():
    partition = panset[part]
    for seq_id in partition.keys():
        sample = partition[seq_id]
        print(len(sample['seq']))
        print(len(sample['coord']))
        print(len(sample['coord_sq']))
        print(len(sample['flip']))
        break
    break

200
200
200
200


making a dataframe for vectorized operations

In [4]:
panset_df = pd.DataFrame(columns=['seq_id', 'seq', 'coord', 'coord_sq', 'flip', 'label'])

flat_data = []
for part in panset.keys():
    for seq_id, data in panset[part].items():
        row = {'part': part, 'seq_id': seq_id, **data}
        flat_data.append(row)

# Convert to DataFrame
df = pd.DataFrame(flat_data)

# del column part
del df['part']
#turn seq_id into index
df.set_index('seq_id', inplace=True)

# remove the coord, coord_sq, flip columns
del df['coord']
del df['coord_sq']
del df['flip']

# preview the data
df.head()

,seq,label
seq_id,,
81,CCCCTGAGCTAGCCATGCTCTGACAGTCTCAGTTGCACACACGAGC...,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
146,TCTCGGTCAGGAACGGTCCCGGGCCTCCCGCCCGCCTCCCTCCAGC...,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
147,CCGCCCGCGCCTCCGGGTCCTAACGCCGCCGCTCGCCCTCCACTGC...,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
449,CAAAGTGCTAAGTATGGTAGATTGCAAACATAAGTGGCCACATAAT...,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
1178,AGCTGACTCATGTTGGGGGCAGCCCACTCACAGTGGCCCTGACCCA...,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."


In [31]:
# copy df
df_copy = df.copy()

# load tf index
tf_index = torch.load(tf_index_file)

# preview the data
df_copy.head()

,seq,label
seq_id,,
81,CCCCTGAGCTAGCCATGCTCTGACAGTCTCAGTTGCACACACGAGC...,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
146,TCTCGGTCAGGAACGGTCCCGGGCCTCCCGCCCGCCTCCCTCCAGC...,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
147,CCGCCCGCGCCTCCGGGTCCTAACGCCGCCGCTCGCCCTCCACTGC...,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
449,CAAAGTGCTAAGTATGGTAGATTGCAAACATAAGTGGCCACATAAT...,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
1178,AGCTGACTCATGTTGGGGGCAGCCCACTCACAGTGGCCCTGACCCA...,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."


loading bubbles and standardizing the df to only have bubble-having sequences

In [6]:
# load the bubble_data_overall pkl file
bubble_data_overall_file = '/project2/xinhe/waqaas/DNA-breathing/data/chipseq_features/bubble_data/bubble_data_overall.pkl'

In [61]:
with open(bubble_data_overall_file, 'rb') as f:
    bubble_data_overall = pickle.load(f)

# TF logic now begins

In [17]:
selected_tfs = ['POU2F2', 'CTCF', 'ETS1', 'JUND', 'MAFK', 'EBF1', 'PAX5']

In [25]:
def check_max(row, n, max_index):
    if row['tf_cell_lines'][max_index] == 1:
        row['tf_label'][n] = 1
    return row

def tf_cell_line_sprucer(df, tf_index, selected_tfs):
    # create empty list of len(selected_tfs)
    df['tf_label'] = [ [0]*len(selected_tfs) for _ in range(len(df)) ]
    for n, tf in enumerate(selected_tfs):
        indices = tf_index[tf]
        df['tf_cell_lines'] = df['label'].apply(lambda x: x[indices])
        # turn into np array
        df['tf_cell_lines'] = df['tf_cell_lines'].apply(np.array)
        # vertical stack
        sums = np.vstack(df['tf_cell_lines'])
        sums = sums.sum(axis=0)
        max_index = np.argmax(sums)
        df = df.apply(check_max, axis=1, args=(n, max_index,))
        del df['tf_cell_lines']
    return df

In [32]:
df_copy = tf_cell_line_sprucer(df_copy, tf_index, selected_tfs)

In [35]:
# save the df_copy to a pkl file
df_copy.to_pickle('/project2/xinhe/waqaas/DNA-breathing/data/Chipseq_data/seq_breathing_feat_tf_labelled.pkl')

In [53]:
# Step 2: Filter Sequences for Selected TFs
# keep the rows in which there is a 1 in df['tf_label']
filtered_data = df_copy[df_copy['tf_label'].apply(lambda x: 1 in x)]

In [54]:
# get shapes
print(df_copy.shape)
print(filtered_data.shape)

(886625, 3)
(227971, 3)


In [57]:
final_df = pd.DataFrame()

for i in range(len(selected_tfs)):
    positive = filtered_data[filtered_data['tf_label'].apply(lambda x: x[i] == 1)].head(3000)
    negative = filtered_data[filtered_data['tf_label'].apply(lambda x: x[i] == 0)].head(3000)
    final_df = pd.concat([final_df, positive, negative])

final_df = final_df[~final_df.index.duplicated(keep='first')]

In [59]:
# get shapes
print(final_df.shape)

(17276, 3)


# Removing already done sequences

In [62]:
# remove the sequences from final_dataset that have already been analysed
for key, value in bubble_data_overall.items():
    final_df = final_df.drop(key, errors='ignore')

In [70]:
print(final_df.shape)

(15125, 3)


In [66]:
# load up the sequences.txt file
sequences_backup_file = '/project2/xinhe/waqaas/DNA-breathing/data/chipseq_features/sequence_file/sequences.txt'

In [71]:
with open(sequences_backup_file, 'r') as f:
    sequences_backup = f.read().splitlines()
    print(len(sequences_backup))
    # only keep the sequences that are in final_dataset
    sequences_backup = [seq for seq in sequences_backup if seq.split()[0] in final_df.index]

print(len(sequences_backup))

877680
14868


In [72]:
# save the sequences_backup.txt file
sequences_backup_file = '/project2/xinhe/waqaas/DNA-breathing/data/chipseq_features/sequence_7tf_subset/sequences_subset.txt'

with open(sequences_backup_file, 'w') as f:
    for seq in sequences_backup:
        f.write(seq + '\n')

In [60]:
for tf in selected_tfs:
    print(tf)
    print(final_df[final_df['label'].apply(lambda x: tf_presence(x, tf_index[tf])) == 1].shape)
    print(final_df[final_df['label'].apply(lambda x: tf_presence(x, tf_index[tf])) == 0].shape)
    print()

POU2F2
(3794, 3)
(13482, 3)

CTCF
(7063, 3)
(10213, 3)

ETS1
(3135, 3)
(14141, 3)

JUND
(3368, 3)
(13908, 3)

MAFK
(3275, 3)
(14001, 3)

EBF1
(3515, 3)
(13761, 3)

PAX5
(3189, 3)
(14087, 3)

